The Following Code works on Google Colab and uploading on Tweets.csv file

1.Import the libraries, load dataset, print shape of data, data description.



In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('Tweets.csv')
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [3]:
data.shape

(14640, 15)

In [4]:
data.describe()

,tweet_id,airline_sentiment_confidence,negativereason_confidence,retweet_count
count,1.464000e+04,14640.000000,10522.000000,14640.000000
mean,5.692184e+17,0.900169,0.638298,0.082650
std,7.791112e+14,0.162830,0.330440,0.745778
min,5.675883e+17,0.335000,0.000000,0.000000
25%,5.685592e+17,0.692300,0.360600,0.000000
50%,5.694779e+17,1.000000,0.670600,0.000000
75%,5.698905e+17,1.000000,1.000000,0.000000
max,5.703106e+17,1.000000,1.000000,44.000000


# **2.Understand of data-columns:**

In [5]:
data.columns

Index(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline',
       'airline_sentiment_gold', 'name', 'negativereason_gold',
       'retweet_count', 'text', 'tweet_coord', 'tweet_created',
       'tweet_location', 'user_timezone'],
      dtype='object')

a.Drop all other columns except “text” and “airline_sentiment”.

In [6]:
data.drop(data.columns.difference(['text','airline_sentiment']), 1, inplace=True)

In [7]:
data.shape

(14640, 2)

In [8]:
data.head()

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


# **3. Text pre-processing: Data preparation.**

In [9]:
# imports
from bs4 import BeautifulSoup
# function to remove HTML tags
def remove_html_tags(text):
    return BeautifulSoup(text, 'html.parser').get_text()

In [10]:
# Rwmove html tags
for i in range(len(data['text'])):
  data['text'][i] = remove_html_tags(data['text'][i])

In [11]:
#conda install -c anaconda nltk
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> l
Packages:
  [ ] abc................. Australian Broadcasting Commission 2006
  [ ] alpino.............. Alpino Dutch Treebank
  [ ] averaged_perceptron_tagger Averaged Perceptron Tagger
  [ ] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [ ] basque_grammars..... Grammars for Basque
  [ ] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
                           Extraction Systems in Biology)
  [ ] bllip_wsj_no_aux.... BLLIP Parser: WSJ Model
  [ ] book_grammars....... Grammars from NLTK Book
  [ ] brown............... Brown Corpus
  [ ] brown_tei........... Brown Corpus (TEI XML Version)
  [ ] cess_cat............ CESS-CAT Treebank
  [

True

In [12]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
from nltk.tokenize import word_tokenize
token = []
for i in range(len(data['text'])):
  token.append(word_tokenize(data['text'][i]))

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
for i in range(len(data['text'])):
  filtered_sentence = [w for w in token[i] if not w in stop_words] 
  token[i] = filtered_sentence

In [16]:
import string
# remove all tokens that are not alphabetic
words = []
for i in range(len(data['text'])):
  words.append([word for word in token[i] if word.isalpha()])
  for j in range(len(words[i])):
    words[i][j] = words[i][j].lower()

In [17]:
from nltk.stem import LancasterStemmer,PorterStemmer,SnowballStemmer
Pst = PorterStemmer()

In [18]:
for i in range(len(words)):
  for j in range(len(words[i])):
    words[i][j] = Pst.stem(words[i][j])

In [20]:
data2 = data
for i in range(len(words)):
  a = ""
  for j in range(len(words[i])):
     a=a + words[i][j] + " "
  data2['text'][i] = a

In [21]:
data2.head() #After Preprocessing

,airline_sentiment,text
0,neutral,virginamerica what dhepburn said
1,positive,virginamerica plu ad commerci experi tacki
2,neutral,virginamerica i today must mean i need take an...
3,negative,virginamerica realli aggress blast obnoxi ente...
4,negative,virginamerica realli big bad thing


In [22]:
X = data2['text']
Y = data2['airline_sentiment']

4.    Vectorization
5.    Fit and evaluate model using both type of vectorization.

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(lowercase=True, stop_words='english', min_df=2)
X_cv = cv.fit_transform(X.values.astype('U'))
X_cv.shape
X_names = cv.get_feature_names()
X_cv_df = pd.DataFrame(X_cv.toarray(),columns = X_names)
X_cv_df.describe()

,aa,aadv,aadvantag,aafail,aarp,abandon,abc,abcnetwork,abil,abl,aboard,abq,abroad,absolut,absorb,absurd,abt,abus,abysm,ac,acc,accept,access,accid,accident,accommod,accompani,accomplish,accord,accordingli,account,accru,acct,accur,accus,achiev,acknowledg,acquir,acquisit,act,...,xna,xoxo,xt,xx,ya,yall,yard,yay,ye,yea,yeah,year,yearli,yell,yep,yest,yesterday,yike,yo,york,youareonyourown,young,younger,yousuck,youth,youv,yow,yr,yuck,yuma,yummi,yup,yvonn,yvr,yyz,zero,zkatcher,zone,zoom,zurich
count,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,...,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000
mean,0.013934,0.000137,0.000683,0.000137,0.000205,0.000273,0.000546,0.000205,0.000478,0.008265,0.000273,0.000615,0.000137,0.003552,0.000137,0.000956,0.000478,0.000205,0.000410,0.000273,0.000137,0.003757,0.002801,0.000410,0.000273,0.002391,0.000137,0.000273,0.000751,0.000273,0.005806,0.000137,0.000820,0.000751,0.000137,0.000205,0.000546,0.000137,0.000137,0.001230,...,0.000205,0.000137,0.000137,0.000137,0.000820,0.002186,0.000137,0.000751,0.016462,0.000820,0.003074,0.009563,0.000137,0.001025,0.001161,0.000273,0.007240,0.000137,0.000956,0.000820,0.000137,0.000615,0.000137,0.000273,0.000205,0.000137,0.000137,0.002254,0.000137,0.000273,0.000137,0.000615,0.000137,0.000478,0.000956,0.002459,0.000205,0.000683,0.000137,0.000273
std,0.125660,0.011688,0.026127,0.011688,0.014314,0.016528,0.023371,0.014314,0.021862,0.091290,0.016528,0.024787,0.011688,0.059494,0.011688,0.030910,0.021862,0.014314,0.023373,0.016528,0.011688,0.061180,0.052848,0.020241,0.016528,0.048838,0.011688,0.016528,0.027402,0.016528,0.078629,0.011688,0.028619,0.027402,0.011688,0.014314,0.023371,0.011688,0.011688,0.035044,...,0.014314,0.011688,0.011688,0.011688,0.030914,0.048143,0.011688,0.027402,0.131472,0.028619,0.055358,0.100093,0.011688,0.031994,0.034058,0.016528,0.084785,0.011688,0.036950,0.028619,0.011688,0.024787,0.011688,0.016528,0.014314,0.011688,0.011688,0.050224,0.011688,0.016528,0.011688,0.024787,0.011688,0.021862,0.030910,0.049529,0.014314,0.026127,0.011688,0.016528
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
xT, xt, yT, yt = train_test_split(X_cv_df, Y, test_size=0.30, random_state=11)
model = LogisticRegression()
model.fit(xT, yT)
model.predict(xt)
pd.DataFrame(model.predict(xt))
model.score(xT,yT)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8966627634660421

In [25]:
res = pd.DataFrame(model.predict(xt))
res

,0
0,positive
1,negative
2,negative
3,negative
4,positive
...,...
4387,negative
4388,negative
4389,negative
4390,negative


In [26]:
pd.DataFrame(model.predict_proba(xt))

,0,1,2
0,0.012397,0.012299,0.975304
1,0.990027,0.003772,0.006201
2,0.906792,0.066127,0.027081
3,0.998449,0.001496,0.000055
4,0.028212,0.026576,0.945212
...,...,...,...
4387,0.597264,0.162959,0.239777
4388,0.995104,0.004345,0.000551
4389,0.637274,0.021400,0.341326
4390,0.590242,0.244241,0.165517


In [27]:
model.score(xt, yt)

0.7800546448087432

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(lowercase=True, stop_words='english',min_df=2)
tv_1 = tv.fit_transform(X)
tv_1.shape

(14640, 4132)

In [29]:
tv_1_names = tv.get_feature_names()
tv_1_df = pd.DataFrame(tv_1.toarray(),columns= tv_1_names)
tv_1_df.describe()

,aa,aadv,aadvantag,aafail,aarp,abandon,abc,abcnetwork,abil,abl,aboard,abq,abroad,absolut,absorb,absurd,abt,abus,abysm,ac,acc,accept,access,accid,accident,accommod,accompani,accomplish,accord,accordingli,account,accru,acct,accur,accus,achiev,acknowledg,acquir,acquisit,act,...,xna,xoxo,xt,xx,ya,yall,yard,yay,ye,yea,yeah,year,yearli,yell,yep,yest,yesterday,yike,yo,york,youareonyourown,young,younger,yousuck,youth,youv,yow,yr,yuck,yuma,yummi,yup,yvonn,yvr,yyz,zero,zkatcher,zone,zoom,zurich
count,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,...,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000
mean,0.004016,0.000055,0.000285,0.000068,0.000093,0.000118,0.000259,0.000094,0.000213,0.002949,0.000124,0.000262,0.000070,0.001492,0.000065,0.000435,0.000193,0.000094,0.000184,0.000122,0.000062,0.001419,0.001028,0.000202,0.000108,0.000935,0.000051,0.000135,0.000326,0.000135,0.001975,0.000060,0.000333,0.000318,0.000052,0.000107,0.000254,0.000051,0.000057,0.000524,...,0.000115,0.000090,0.000061,0.000097,0.000401,0.000922,0.000061,0.000398,0.006038,0.000408,0.001238,0.003068,0.000068,0.000413,0.000561,0.000123,0.002565,0.000099,0.000351,0.000315,0.000071,0.000263,0.000062,0.000144,0.000085,0.000067,0.000068,0.000762,0.000068,0.000160,0.000073,0.000302,0.000055,0.000186,0.000409,0.000879,0.000108,0.000275,0.000054,0.000110
std,0.036735,0.004724,0.010934,0.005821,0.006673,0.007155,0.011356,0.006572,0.009940,0.033732,0.007666,0.010746,0.005947,0.026527,0.005588,0.014169,0.008947,0.006601,0.010194,0.007464,0.005279,0.023858,0.019751,0.010172,0.006582,0.019564,0.004353,0.008304,0.012096,0.008368,0.026947,0.005177,0.011768,0.011752,0.004459,0.007518,0.011080,0.004387,0.004895,0.015264,...,0.008034,0.007883,0.005238,0.008791,0.015220,0.020653,0.005191,0.015043,0.050866,0.014628,0.023071,0.032971,0.005801,0.013162,0.016983,0.007440,0.030961,0.008903,0.012784,0.011052,0.006111,0.010760,0.005307,0.008762,0.005944,0.005749,0.005937,0.016880,0.005830,0.009790,0.006221,0.012652,0.004679,0.008575,0.013438,0.017880,0.007657,0.010546,0.004600,0.006662
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [30]:
x_train,x_test,y_train,y_test = train_test_split(tv_1_df,Y,test_size = 0.30, random_state = 11)
model.fit(x_train,y_train)
ypredict = model.predict(x_test)
model.score(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8534348165495707

In [31]:
model.score(x_test, y_test)

0.788023679417122

In [32]:
Algo = []
train = []
test = []
recall = []
roc = []

In [33]:
from sklearn.metrics import recall_score,precision_score,roc_auc_score
Algo.append('Logistic Regression')
train.append(model.score(x_train,y_train))
test.append(model.score(x_test,y_test))
#recall.append(recall_score(ypredict,y_test),average = 'micro')
#roc.append(roc_auc_score(ypredict,y_test))


In [34]:
from sklearn.tree import DecisionTreeClassifier

In [35]:
model_decisiontree = DecisionTreeClassifier()
model_decisiontree.fit(x_train,y_train)
ypredict_decisiontree = model_decisiontree.predict(x_test)
print(model_decisiontree.tree_.max_depth)
#model_decisiontree.score(x_train,y_train) Overfitting of training set
#model_decisiontree.score(x_test,y_test)

206


In [36]:
model_pruned = DecisionTreeClassifier(criterion='entropy',max_depth=3,random_state=5)

In [37]:
model_pruned.fit(x_train,y_train)
ypredict = model_pruned.predict(x_test)

In [38]:
Algo.append('Decision Tree using Entropy')
train.append(model_pruned.score(x_train,y_train))
test.append(model_pruned.score(x_test,y_test))
#recall.append(recall_score(ypredict,y_test))
#roc.append(roc_auc_score(ypredict,y_test))

In [39]:
model = DecisionTreeClassifier(criterion='gini',max_depth=3,random_state=5)
#model = DecisionTreemodel()
model.fit(x_train,y_train)
ypredict = model.predict(x_test)
Algo.append('Decision Tree using Gini')
train.append(model.score(x_train,y_train))
test.append(model.score(x_test,y_test))
#recall.append(recall_score(ypredict,y_test))
#roc.append(roc_auc_score(ypredict,y_test))

In [40]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 100)
model.fit(x_train,y_train)
ypredict = model.predict(x_test)
Algo.append('Random Forest')
train.append(model.score(x_train,y_train))
test.append(model.score(x_test,y_test))

Following Classifier have low accuracy and are time consuming hence commented

In [41]:
#from sklearn.ensemble import BaggingClassifier
#model = BaggingClassifier(n_estimators=100,max_samples=0.7,bootstrap=True,oob_score=True,random_state=11)
#model.fit(x_train,y_train)
#ypredict = model.predict(x_test)
#Algo.append('Bagging model')
#train.append(model.score(x_train,y_train))
#test.append(model.score(x_test,y_test))
#recall.append(recall_score(ypredict,y_test))
#roc.append(roc_auc_score(ypredict,y_test))

In [42]:
#from sklearn.ensemble import AdaBoostClassifier
#model = AdaBoostClassifier(n_estimators=200,learning_rate=0.1,random_state=11)
#model.fit(x_train,y_train)
#ypredict = model.predict(x_test)
#Algo.append('AdaBoost model')
#train.append(model.score(x_train,y_train))
#test.append(model.score(x_test,y_test))
#recall.append(recall_score(ypredict,y_test))
#roc.append(roc_auc_score(ypredict,y_test))

In [43]:
#from sklearn.ensemble import GradientBoostingClassifier
#model = GradientBoostingClassifier(n_estimators=200,random_state=11)
#model.fit(x_train,y_train)
#ypredict = model.predict(x_test)
#Algo.append('Gradient Boosting model')
#train.append(model.score(x_train,y_train))
#test.append(model.score(x_test,y_test))
#recall.append(recall_score(ypredict,y_test))
#roc.append(roc_auc_score(ypredict,y_test))

In [44]:
results = pd.DataFrame()
results['Model'] = Algo
results['Training Accuracy'] = train
results['Testing Accuracy'] = test
#results['Recall'] = recall
#results['ROC AUC Score'] = roc
results = results.set_index('Model')
results

,Training Accuracy,Testing Accuracy
Model,,
Logistic Regression,0.853435,0.788024
Decision Tree using Entropy,0.678962,0.681922
Decision Tree using Gini,0.683743,0.686931
Random Forest,0.993657,0.757058


6. The preprocessing of the text data is an essential step as it makes the raw text ready for mining, i.e., it becomes easier to extract information from the text and apply machine learning algorithms to it. If we skip this step then there is a higher chance that you are working with noisy and inconsistent data.Vectorization is to Convert the data into Numerical Matrix in which ML Algorithms can easily be applied

In this Case we find Logistic Regression to be most effective with 78.8% Accuracy

In [45]:
for i in range(len(Y)):
  if Y[i] == 'positive':
    Y[i] = 0.0
  if Y[i] == 'negative':
    Y[i] = 1.0
  if Y[i] == 'neutral':
    Y[i] = 2.0

# **7.    If applicable apply any Classification based Machine learning/Deep Learning framework to have better understanding of the Use Case.**

In [46]:
clf = RandomForestClassifier(
    n_estimators=50,
    criterion='gini',
    max_depth=5,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='auto',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    bootstrap=True,
    oob_score=False,
    n_jobs=-1,
    random_state=0,
    verbose=0,
    warm_start=False,
    class_weight='balanced'
)

In [47]:
#from sklearn.model_selection import StratifiedKFold
#cv = StratifiedKFold(n_splits=5, random_state=123, shuffle=True)
#results = pd.DataFrame(columns=['training_score', 'test_score'])
#fprs, tprs, scores = [], [], []
    
#for (train1, test1), i in zip(cv.split(X_cv_df, Y), range(5)):
#    clf.fit(X_cv_df.iloc[train1], Y.iloc[train1])
#    train.append(clf.score(x_train,y_train))
#    test.append(clf.score(x_test,y_test))
#    Algo.append('RFCV')
#train.append(model.score(x_train,y_train))

In [48]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
ts_split= TimeSeriesSplit(n_splits=5)
for train_index, test_index in ts_split.split(X_cv_df):
        X_train, X_test = X_cv_df[:len(train_index)], X_cv_df[len(train_index): (len(train_index)+len(test_index))]
        y_train, y_test = Y[:len(train_index)].values.ravel(), Y[len(train_index): (len(train_index)+len(test_index))].values.ravel()

In [49]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12200 entries, 0 to 12199
Columns: 4132 entries, aa to zurich
dtypes: int64(4132)
memory usage: 384.6 MB


In [50]:
X_train =np.array(X_train)
X_test =np.array(X_test)

X_tr_t = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_tst_t = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

In [51]:
for i in range(len(y_train)):
  if y_train[i] == 'positive':
    y_train[i] = 0.0
  if y_train[i] == 'negative':
    y_train[i] = 1.0
  if y_train[i] == 'neutral':
    y_train[i] = 2.0
for i in range(len(y_test)):
  if y_test[i] == 'positive':
    y_test[i] = 0.0
  if y_test[i] == 'negative':
    y_test[i] = 1.0
  if y_test[i] == 'neutral':
    y_test[i] = 2.0
y_train

array([2.0, 0.0, 2.0, ..., 0.0, 1.0, 0.0], dtype=object)

In [52]:
X_train = np.asarray(X_train).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)
X_tr_t = np.asarray(X_tr_t).astype(np.float32)
X_tst_t = np.asarray(X_tst_t).astype(np.float32)

In [53]:
#import tensorflow as tf
#from tensorflow import keras
from sklearn.metrics import mean_squared_error, r2_score
from keras.layers import LSTM,Dense,Dropout
from keras.models import Sequential
from keras.layers import Dense
import keras.backend as K
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import load_model
from keras.layers import LSTM
K.clear_session()
model_lstm = Sequential()
model_lstm.add(LSTM(16, input_shape=(1, X_train.shape[1]), activation='relu', return_sequences=False))
model_lstm.add(Dense(1))
model_lstm.compile(loss='mean_squared_error', optimizer='adam')
early_stop = EarlyStopping(monitor='loss', patience=5, verbose=1)
history_model_lstm = model_lstm.fit(X_tr_t, y_train, epochs=25, batch_size=8, verbose=1, shuffle=False, callbacks=[early_stop])

Epoch 1/25
1525/1525 [==============================] - 5s 3ms/step - loss: 0.4006
Epoch 2/25
1525/1525 [==============================] - 5s 3ms/step - loss: 0.2578
Epoch 3/25
1525/1525 [==============================] - 5s 3ms/step - loss: 0.2221
Epoch 4/25
1525/1525 [==============================] - 5s 3ms/step - loss: 0.1941
Epoch 5/25
1525/1525 [==============================] - 5s 3ms/step - loss: 0.1701
Epoch 6/25
1525/1525 [==============================] - 5s 3ms/step - loss: 0.1496
Epoch 7/25
1525/1525 [==============================] - 5s 3ms/step - loss: 0.1333
Epoch 8/25
1525/1525 [==============================] - 5s 3ms/step - loss: 0.1200
Epoch 9/25
1525/1525 [==============================] - 5s 3ms/step - loss: 0.1093
Epoch 10/25
1525/1525 [==============================] - 5s 3ms/step - loss: 0.1005
Epoch 11/25
1525/1525 [==============================] - 5s 3ms/step - loss: 0.0926
Epoch 12/25
1525/1525 [==============================] - 5s 3ms/step - loss: 0.0868
E

In [55]:
y_pred_test_lstm = model_lstm.predict(X_tst_t)
y_train_pred_lstm = model_lstm.predict(X_tr_t)
print("The R2 score on the Train set is:\t{:0.3f}".format(r2_score(y_train, y_train_pred_lstm)))
r2_train = r2_score(y_train, y_train_pred_lstm)

The R2 score on the Train set is:	0.883
